Меликбекян Ашот Арсенович

Импорт библиотек

In [ ]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.decomposition import TruncatedSVD

Загрузка и предобработка данных

In [ ]:
artist_list = pd.read_csv('/content/drive/MyDrive/МИРЭА/AI/lastfm_artist_list.csv')
user_scrobble = pd.read_csv('/content/drive/MyDrive/МИРЭА/AI/lastfm_user_scrobbles.csv')

In [ ]:
artist_list.head()

,artist_id,artist_name
0,1,__Max__
1,2,_Algol_
2,3,-123 Min.
3,4,-Oz-
4,5,-T De Sangre


In [ ]:
user_scrobble.head()

,user_id,artist_id,scrobbles
0,1,4562,13883
1,1,10191,11690
2,1,494,11351
3,1,6673,10300
4,1,8402,8983


In [ ]:
merged = pd.merge(artist_list, user_scrobble, on='artist_id')
merged.head()

,artist_id,artist_name,user_id,scrobbles
0,1,__Max__,520,55
1,2,_Algol_,416,60
2,3,-123 Min.,1776,73
3,4,-Oz-,73,793
4,4,-Oz-,1562,461


Топ-10 популярных исполнителей

In [ ]:
pd.DataFrame(merged.groupby('artist_name')['scrobbles'].count().sort_values(ascending=False)).head(15)

,scrobbles
artist_name,
Lady Gaga,611
Britney Spears,522
Rihanna,485
The Beatles,480
Katy Perry,473
Madonna,429
Avril Lavigne,417
Christina Aguilera,409
Muse,400


Создание опорной таблицы

In [ ]:
crosstab = merged.pivot_table(values='scrobbles', index='user_id', columns='artist_name', fill_value=0)
crosstab.head()

artist_name,!!!,!Deladap,!Distain,#####,#2 Orchestra,$Lick,$Lick Feat. Six-Fo,(1962-1966),(Hed) Planet Earth,(Pre) Kingdom,(Sic)Monic,(We Are) Performance,*Nsync,+\-,-123 Min.,-Oz-,-T De Sangre,...And The Earth Swarmed With Them,...And You Will Know Us By The Trail Of Dead,.38 Special,.Crrust,06 Children Of The Cross,1-800-Zombie,10 Years,10-20 Без Выходных,100 Bitches,1000names,100blumen,100dbs,100deadrabbits!!!,10cc,11.43,110,112,1125,12 Rounds,12 Stones,120 Days,1200 Micrograms,12012,...,미인도(美人圖),민호,박정현,반야,배치기,백지영,비,빅뱅,샤이니,서태지,소녀시대,소녀시대 ＆ 2pm,손담비,송지은,승리,시크릿,신화,아웃사이더,아이비,애프터 스쿨(After School),엠블랙(Mblaq),원더걸스,윤하,이박사,이수영(여),이정현,이효리,장근석,제빵왕,태양,티아라,현아,Ｂｏｏｇｉｅｍａｎ,Ｂｏｒｎ,ＬｕＬｕ,ＭＡＮＮＥＱＵＩＮ,Ｍａｃｈｉｎｅ,ＰＥＲＥＳＴＲＯＩＫＡ,ＲＥＤ,ＶａｎｅｓｓＡ
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
X = crosstab.T
SVD = TruncatedSVD(n_components=12, random_state=5)
resultant_matrix = SVD.fit_transform(X)
resultant_matrix.shape

(17492, 12)

In [ ]:
corr_mat = np.corrcoef(resultant_matrix)
corr_mat.shape

(17492, 17492)

In [ ]:
def get_similar_artists(artist_name: str, amount: int=5):
  '''Рекомендует похожих артистов.
     
  Parametrs
  ---------
  artist_name: str
      имя артиста
  amount: int = 5
      количество рекомендаций

  Returns
  ----------
      DataFrame похожих артистов
  '''

  col_idx = crosstab.columns.get_loc(artist_name)
  corr_specific = corr_mat[col_idx]
  df = pd.DataFrame({'corr_specific': corr_specific, 'artist_name': crosstab.columns})  \
    .sort_values('corr_specific', ascending=False) \
    .iloc[1:] \
    .head(amount)
  return df

In [ ]:
get_similar_artists('Nirvana', 10)

,corr_specific,artist_name
12757,0.914338,Scorpions
2682,0.896744,Carpark North
16149,0.894030,Violins
10563,0.893471,Neuroticfish
4690,0.892670,Eight Legs
7913,0.884165,Karjalan Sissit
13304,0.882082,Smolik
10805,0.878867,Nox Arcana
16651,0.878086,Your Vegas
5041,0.876782,Eths


In [ ]:
get_similar_artists('Eminem', 10)

,corr_specific,artist_name
7279,0.997725,Jay-Z
2418,0.997518,Bruno Mars
4031,0.992728,Dev
4933,0.991169,Enrique Iglesias
7900,0.990357,Kardinal Offishall
1955,0.989200,Black Eyed Peas
9444,0.988726,Mario Vazquez
13048,0.986867,Shreya Ghoshal & Shaan
7846,0.986867,K.K.
2575,0.986867,Cameron Cartio


In [ ]:
get_similar_artists('Rihanna', 10)

,corr_specific,artist_name
13977,0.991099,Taio Cruz
7980,0.988592,Katy Perry
8008,0.978527,Ke$Ha
3154,0.969695,Claudia Leitte
6227,0.967417,Gwen Stefani
2998,0.967288,Chris Brown
7275,0.967180,Jay Sean
1006,0.960957,Anna Netrebko
2579,0.957678,Camille
15964,0.957190,Usher


In [ ]:
get_similar_artists('The Beatles', 10)

,corr_specific,artist_name
1698,0.979164,Belchior
4653,0.979164,Ednardo
3760,0.976349,David Nevue
8079,0.976349,Kevin Johansen
11065,0.976349,Oswaldo Montenegro
5284,0.976349,Felpeyu
6212,0.973592,Gustavo Garde
4171,0.971442,Dire Straits
13856,0.967995,Supertramp
8551,0.965871,Led Zeppelin


In [ ]:
def get_familiar_artists(user_id: int, amount: int=5):
  '''Находит знакомых артистов
     
  Parametrs
  ---------
  user_id: int = 5
      id пользователя

  Returns
  ----------
      Список знакомых артистов
  '''

  user = crosstab.iloc[[user_id]]
  all_artists = user.loc[:, (user != 0).any(axis=0)].reset_index().T
  all_artists.columns = ['rating']
  all_artists = all_artists[1:]
  familiar_artists = list(all_artists.sort_values('rating', ascending=False).head().index)
  return familiar_artists

Делаем рекомендации 10 первым пользователям.

Для каждого пользователя собираем знакомых ему артистов, делаем по каждому знакомому артисту рекомендации, собираем матрицу корреляций со всеми артистами и рекомендуем тех, у кого больше корреляция 

In [ ]:
dct = {}
indexes = list(crosstab.index)
for user_id in indexes[:1000]:
  familiar_artists = get_familiar_artists(user_id, 5)
  recs = list(map(get_similar_artists, familiar_artists))
  top_recs = pd.DataFrame()
  for rec in recs:
    top_recs = top_recs.append(rec)
  top_recs = list(top_recs.sort_values(by='corr_specific', ascending=False).drop_duplicates().head().artist_name)
  dct[f'user_{user_id}'] = top_recs

Рекомендации для user_990

In [ ]:
dct['user_990']

['Niccolo Paganini',
 'Royal Stockholm Philharmonic Orchestra',
 'Freenote',
 'Ella Fitzgerald & Louis Armstrong',
 'M-Flo']